## 1. Set-up

In [1]:
# Mount Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
MODEL_VERSION = "multimodal_EN"

In [3]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.0 MB/s eta 0:00:0

In [4]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset,load_from_disk, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer, Trainer, VisualBertModel
from scipy.stats import pearsonr
import sklearn.metrics
from datasets import load_metric, load_dataset
from sklearn.metrics import mean_squared_error

## 2. Arrange datasets and tokenize them

In this section, I'll use the dataset provided from

In [5]:
column_names = ["sentence1", "sentence2", "label"]
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/data/en-test.txt", names=column_names, sep="\t")
validation_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/data/en-val.txt", names=column_names, sep="\t")
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/data/en-train.txt", names=column_names, sep="\t")

In [6]:
train_df

,sentence1,sentence2,label
0,umm the california was a cargo ship i dont car...,the california was a cargo ship .,4.00
1,how can i prepare this old exterior wall for p...,how do i prepare this exterior concrete wall f...,4.00
2,the man is playing the guitar .,a man is playing guitar .,5.00
3,`` it was a final test before delivering the m...,state radio said it was the last test before t...,4.00
4,"this does not fully answer your question , but...","i am a phd student in computational science , ...",0.00
...,...,...,...
13360,"pictures of the day : pakistan , syria , indon...",pictures of the day : pakistan and elsewhere,3.20
13361,claims about trayvon 's character and when did...,slanted blogs and when did i do that ?,2.60
13362,msn messenger 6 will be available for download...,the msn messenger 6 software will be available...,3.25
13363,and they never wondr why ! ! u.s gets it wrong...,"opinion : u.s gets it wrong on egypt , again",4.40


In [ ]:
#dataset["test"]["sentence1"][:5]

In [7]:
dataset_train = Dataset.from_pandas(train_df,preserve_index=False) 
dataset_dev = Dataset.from_pandas(validation_df,preserve_index=False) 
dataset_test = Dataset.from_pandas(test_df,preserve_index=False) 
dataset = DatasetDict({"train": dataset_train,
                     "test": dataset_test,
                     "validation": dataset_dev})
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 13365
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 250
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 1500
    })
})

Tokenize datasets

In [8]:
#model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)
model = BertForSequenceClassification.from_pretrained("uclanlp/visualbert-vqa-coco-pre", num_labels=1)
#model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre", num_labels=1)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", model_max_length=512)

You are using a model of type visual_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing BertForSequenceClassification: ['visual_bert.encoder.layer.0.attention.output.LayerNorm.bias', 'visual_bert.encoder.layer.1.output.dense.bias', 'visual_bert.encoder.layer.11.output.dense.bias', 'visual_bert.encoder.layer.0.attention.self.query.bias', 'visual_bert.encoder.layer.0.attention.self.key.bias', 'visual_bert.embeddings.LayerNorm.bias', 'visual_bert.encoder.layer.3.output.LayerNorm.bias', 'visual_bert.encoder.layer.7.intermediate.dense.bias', 'visual_bert.encoder.layer.2.attention.self.query.bias', 'visual_bert.encoder.layer.5.intermediate.dense.weight', 'visual_bert.encoder.layer.4.intermediate.dense.weight', 'visual_bert.encoder.layer.7.attention.self.key.weight', 'visual_bert.encoder.layer.6.attention.self.value.bias', 'visual_bert.encoder.layer.10.attention.self.key.bias', 'visual_bert.encoder.layer.10.output.dense.bias', 'visual_bert.embeddings.token_type_embeddings.weig

In [9]:
# examples = [("monitos en la cama", "estamos en la cama"), ("tres tristes tigres", "los tigres son lindos")]

# tokenizer(examples, padding="max_length", truncation=True)

In [10]:
# print(dataset['train']['sentence1'][20])
# print(dataset['train']['sentence2'][20])

# a = tokenizer(dataset['train']['sentence1'][20], dataset['train']['sentence2'][20], padding=True, truncation=True)
# tokenizer.decode(a["input_ids"])

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/13365 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets

In [ ]:
for input_idd in tokenized_datasets["train"]["input_ids"]:
  if len(input_idd) != 512:
    print(len(input_idd))

We use rmse as a metric since we are in a regression scenario
 (the lower this value, the better the model performs)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## 3. Train the model

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  #evaluation_strategy="steps",
                                  eval_steps = 10,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs = 5,
                                  save_total_limit = 2,
                                  #save_strategy = 'steps',
                                  save_strategy = 'epoch',
                                  #load_best_model_at_end=False
                                  load_best_model_at_end=True)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13365
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4180
  Number of trainable parameters = 109483009


Epoch,Training Loss,Validation Loss,Rmse
1,2.403700,2.123762,1.457313
2,1.861300,2.242557,1.497517
3,1.540200,2.373745,1.540696
4,1.268700,2.678588,1.636639
5,1.068500,2.720738,1.649466


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sentence2, sentence1 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/multimodal_EN/checkpoint-836
Configuration saved in /content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/multimodal_EN/checkpoint-836/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/multimodal_EN/checkpoint-836/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1. If sen

TrainOutput(global_step=4180, training_loss=1.628477762760728, metrics={'train_runtime': 6187.5688, 'train_samples_per_second': 10.8, 'train_steps_per_second': 0.676, 'total_flos': 1.75822384091904e+16, 'train_loss': 1.628477762760728, 'epoch': 5.0})

## 4. Save model

In [ ]:
# model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_{MODEL_VERSION}")
# tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")

In [17]:

# # load the model/tokenizer

model = BertForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}/checkpoint-4180")
# tokenizer = AutoTokenizer.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}")



## 5. New predictions

In [18]:
tokenized_test_dataset = tokenized_datasets["test"]

In [19]:
trainer = Trainer(model=model)

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

def prediction(tokenized_datasets):
    # df=pd.DataFrame({'text':texts})
    # dataset = Dataset.from_pandas(df,preserve_index=False) 
    # tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred)

def evaluate(preds, gold):
    # Compute pearson:
    pearson = pearsonr(preds, gold)[0]
      
    # Compute rmse
    rmse = mean_squared_error(preds, gold, squared=False)

    print(f"RMSE = {rmse:.6f}")
    print(f"Pearson: {pearson:.6f}")

In [20]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 1.649466
Pearson: 0.130906


In [16]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 1.457313
Pearson: 0.054602
